In [1]:
from datasets import load_from_disk
from utils import load_model_and_tokenizer

model_name = "meta-llama/Llama-3.2-3B-Instruct"
model, tokenizer, config = load_model_and_tokenizer(model_name)
finished_dataset = load_from_disk("./generated_data/mr_tydi_tydiqa_final")

Loading model on cuda with torch.float16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import datasets

def format_passages(example):
    """
    Menggabungkan positive_passages dan negative_passages, 
    lalu memformat top 3 passages menjadi string terstruktur.
    """
    # Gabungkan positive_passages dan negative_passages
    all_passages = example["positive_passages"] + example["negative_passages"]
    
    # Format 3 passage pertama
    formatted_passages = [
        f"Judul: {p['title']} \nTeks: {p['text']}" for p in all_passages[:3]  # Ambil 3 passage pertama
    ]
    
    return {"formatted_passages": "\n\n".join(formatted_passages)}

# Terapkan transformasi pada semua split (train, dev, test) dalam sekali `.map()`
processed_dataset = {}

for split in finished_dataset.keys():  # Loop untuk setiap split: "train", "dev", "test"
    processed_dataset[split] = finished_dataset[split].map(format_passages)

# Konversi kembali ke DatasetDict
processed_dataset = datasets.DatasetDict(processed_dataset)

# Cek hasil pada salah satu split
print(processed_dataset["train"][0])  # Lihat hasil dari train


Map:   0%|          | 0/4542 [00:00<?, ? examples/s]

Map:   0%|          | 0/1143 [00:00<?, ? examples/s]

Map:   0%|          | 0/565 [00:00<?, ? examples/s]

{'query_id': '0', 'query': 'dimanakah Dr. Ernest François Eugène Douwes Dekker meninggal?', 'positive_passages': [{'docid': '7080#33', 'text': 'Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.', 'title': 'Ernest Douwes Dekker'}], 'negative_passages': [{'docid': '20439#25', 'text': 'Eduard Douwes Dekker kemudian pindah ke Ingelheim am Rhein dekat Sungai Rhein sampai akhirnya meninggal 19 Februari 1887.', 'title': 'Eduard Douwes Dekker'}, {'docid': '7080#0', 'text': 'Dr. Ernest François Eugène Douwes Dekker (umumnya dikenal dengan nama Douwes Dekker atau Danudirja Setiabudi; ) adalah seorang pejuang kemerdekaan dan pahlawan nasional Indonesia.', 'title': 'Ernest Douwes Dekker'}, {'docid': '7080#2', 'text': 'Douwes Dekker terlahir di Pasuruan, Jawa Timur, pada tanggal 8 Oktober 1879, sebagaimana yang dia tulis pada riwayat hidup singkat saat mendaftar di Universitas Zu

In [ ]:
import random

SEED = 42
random.seed(SEED)

# Ambil sampel dari tiap split
train_sample = random.sample(list(finished_dataset["train"]), 132)
dev_sample = random.sample(list(finished_dataset["dev"]), 40)
test_sample = random.sample(list(finished_dataset["test"]), 28)

# Gabungkan semua sampel
sample_data = train_sample + dev_sample + test_sample

print(f"Total samples: {len(sample_data)}")  # Harusnya 71 (50+13+8)

def combine_passages(example):
    example["all_passages"] = example["positive_passages"] + example["negative_passages"]
    return example

# Terapkan ke semua sampel
sample_data = [combine_passages(ex) for ex in sample_data]

def format_passages(example):
    formatted_passages = [
        f"Judul: {p['title']} \nTeks: {p['text']}" for p in example["all_passages"][:3]  # Ambil 5 passage
    ]
    return "\n\n".join(formatted_passages)  # Gabungkan semua passages

# Tambahkan kolom "formatted_passages"
for ex in sample_data:
    ex["formatted_passages"] = format_passages(ex)

Total samples: 200


In [15]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages'],
        num_rows: 4542
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages'],
        num_rows: 1143
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages'],
        num_rows: 565
    })
})

In [17]:
from summarize import summarize_top_5_combined
import time
from datasets import DatasetDict

model_name = "meta-llama/Llama-3.2-3B-Instruct"

# Dictionary untuk menyimpan hasil per split
processed_splits = {}

# Loop untuk setiap split (train, dev, test)
for split in processed_dataset.keys():
    print(f"🔄 Memproses split: {split}")

    start = time.time()

    # Jalankan rangkuman untuk split tertentu
    processed_split = summarize_top_5_combined(
        model_name=model_name,
        dataset=processed_dataset[split],  # Proses per split
        query_col="query",
        docs_col="formatted_passages", 
        model=model, 
        tokenizer=tokenizer,
        batch_size=1
    )

    end = time.time()
    duration = end - start
    print(f"✅ Split {split} selesai dalam {duration:.2f} detik")

    # Simpan hasil per split ke dalam dictionary
    processed_splits[split] = processed_split

# Gabungkan kembali hasil per split menjadi DatasetDict
final_dataset = DatasetDict(processed_splits)

# Path penyimpanan hasil akhir
save_path = "generated_data/draft_summary_dataset"

# Simpan dataset yang telah digabungkan
final_dataset.save_to_disk(save_path)

print(f"✅ DatasetDict telah disimpan di {save_path}")

🔄 Memproses split: train


Summarizing dataset: 100%|██████████| 4542/4542 [5:51:08<00:00,  4.64s/it]   


✅ Split train selesai dalam 21068.54 detik
🔄 Memproses split: dev


Summarizing dataset: 100%|██████████| 1143/1143 [1:26:45<00:00,  4.55s/it]


✅ Split dev selesai dalam 5206.19 detik
🔄 Memproses split: test


...

✅ Split test selesai dalam 2617.99 detik


Saving the dataset (0/1 shards):   0%|          | 0/4542 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1143 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/565 [00:00<?, ? examples/s]

✅ DatasetDict telah disimpan di generated_data/draft_summary_dataset


In [19]:
loaded_dataset = load_from_disk("./generated_data/draft_summary_dataset")
loaded_dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages', 'summary'],
        num_rows: 4542
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages', 'summary'],
        num_rows: 1143
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'formatted_passages', 'summary'],
        num_rows: 565
    })
})

In [25]:
loaded_dataset['dev'][0]['summary']

'James Hepburn meninggal di Puri Dragsholm, 75 kilometer dari Kopenhagen.'

In [ ]:
# from utils import DeviceType, format_chat_prompt, prepare_inputs
# import torch

# def summarize_single_row(example, model, tokenizer, query_col, docs_col, max_new_tokens=50, temperature=0.7):
#     """
#     Fungsi ini menerima satu baris dataset, lalu menghasilkan rangkuman teks.
    
#     Args:
#         example: Satu instance dari dataset
#         model: Model yang telah dimuat
#         tokenizer: Tokenizer yang telah dimuat
#         query_col: Nama kolom pertanyaan
#         docs_col: Nama kolom dokumen
#         max_new_tokens: Jumlah token yang dihasilkan
#         temperature: Parameter sampling

#     Returns:
#         Dict dengan tambahan kolom "summary"
#     """
#     try:
#         # Ambil teks dari dataset
#         top_5_text = example[docs_col]
#         query = example[query_col]

#         # Siapkan prompt untuk rangkuman
#         messages = [
#             {
#                 "role": "user",
#                 "content": f'{top_5_text}\n\nRingkaslah teks di atas agar dapat menjawab pertanyaan secara mendetail. Jangan berikan pengantar pada hasil. Pertanyaan: "{query}"'
#             }
#         ]

#         formatted_prompt = format_chat_prompt(messages, tokenizer)

#         # Tokenisasi input
#         inputs = prepare_inputs([formatted_prompt], tokenizer, DeviceType.CUDA)
#         token_ids = inputs["input_ids"]

#         # Gunakan model untuk generate teks
#         with torch.no_grad():
#             with torch.amp.autocast('cuda:0'):
#                 summary_output = model.generate(
#                     inputs["input_ids"],
#                     max_new_tokens=max_new_tokens,
#                     temperature=temperature,
#                     top_p=1.0,
#                     do_sample=temperature > 0,
#                     pad_token_id=tokenizer.pad_token_id,
#                     return_dict_in_generate=True
#                 )

#                 # Ambil token hasil generate
#                 generated_ids = summary_output.sequences[0][len(token_ids[0]):]

#                 # Decode hasil keluaran
#                 summary = tokenizer.decode(generated_ids, skip_special_tokens=True)

#         # Kembalikan hasil dengan tambahan kolom summary
#         return {"summary": summary}

#     except Exception as e:
#         print(f"⚠️ Error pada satu instance: {e}")
#         return {"summary": "Error"}
    
# print("num samples:", num_samples)    
# processed_dataset = sample_dataset.select(range(num_samples)).map(
#     lambda x: summarize_single_row(x, model, tokenizer, query_col="query", docs_col="formatted_passages"),
#     batched=False  # Harus False karena kita memproses satu per satu
# )

num samples: 64


Parameter 'function'=<function <lambda> at 0x000002669660EF20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/64 [00:00<?, ? examples/s]